### Load the Libraries and import the data

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import pandas_profiling
import rpy2
import rpy2.rinterface
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")
import seaborn as sns

In [2]:
#Read in the data from the file
df = pd.read_csv('C:/githubrepo/7331_Project/data/Employee_Compensation.csv')
print("Finished Loading Data.")

Finished Loading Data.


### Removal of 'Calendar' from 'Year Type' column
The data was duplicated due to there being two 'Year Type': Fiscal and Calendar. Removing 'Calendar' (correctly) cut the number of rows in the dataset in half and also fixed the 'Department Code' column from having incorrect values. Those values corresponded to a different column that is not in this dataset. See 'Department Code' and 'Department Group Code' columns at the following website to see the difference:

https://data.sfgov.org/City-Management-and-Ethics/Reference-Department-Code-List/j2hz-23ps/data

In [3]:
#Save the dataframe into a new dataframe to clean
df_clean = df
#Keeps all of the Fiscal Year entries
df_clean = df_clean[df_clean['Year Type'] == 'Fiscal']
print(df_clean.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 334535 entries, 2 to 831991
Data columns (total 22 columns):
Year Type                  334535 non-null object
Year                       334535 non-null int64
Organization Group Code    334535 non-null int64
Organization Group         334535 non-null object
Department Code            305591 non-null object
Department                 213202 non-null object
Union Code                 334320 non-null float64
Union                      334320 non-null object
Job Family Code            334535 non-null object
Job Family                 334535 non-null object
Job Code                   334535 non-null object
Job                        334534 non-null object
Employee Identifier        334535 non-null int64
Salaries                   334535 non-null float64
Overtime                   334535 non-null float64
Other Salaries             334535 non-null float64
Total Salary               334535 non-null float64
Retirement                 334535 non

### Fixing 'Year' and 'Job'
Year had one row that had an incorrect year entered (2028). Although the actual value is most likely supposed to be '2018', because this only occurred once and there are more than 300k rows we simply removed it.

There is one value in 'Job' which was null. Because there was no way to determine the true value, and because this only occurred once and there are more than 300k rows we simply removed it.

In [5]:
#Get names of indexes for which column Year has value 2028
indexNames = df_clean.loc[df_clean['Year'] == 2028, 'Year'].index
# Delete these row indexes from dataFrame
df_clean.drop(indexNames, inplace = True)

In [11]:
#Resets the indices for df_clean (just in case)
df_clean = df_clean.reset_index(drop=True)
#Deletes the 1 row missing the Job
indexNames = df_clean.loc[df_clean['Job'].isnull()].index
df_clean.drop(indexNames, inplace = True)

### Removal of rows with negative values
For the continuous columns we have an issue of there being negative value. Having something like a negative salary makes no logical sense and was not explained as being correct in the descriptions of the data. Because of this we removed these values (SEE THE OTHER .IPYNB FOR THE GRAPHS).

In [12]:
#Delete Salaries values with negatives
indexNames = df_clean.loc[df_clean['Salaries'] < 0, 'Salaries'].index
df_clean.drop(indexNames, inplace = True)

In [13]:
#Delete Overtime values with negatives
indexNames = df_clean.loc[df_clean['Overtime'] < 0, 'Overtime'].index
df_clean.drop(indexNames, inplace = True)

In [14]:
#Delete Other Salaries values with negatives
indexNames = df_clean.loc[df_clean['Other Salaries'] < 0, 'Other Salaries'].index
df_clean.drop(indexNames, inplace = True)

In [15]:
#Delete Total Salary values with negatives
indexNames = df_clean.loc[df_clean['Total Salary'] < 0, 'Total Salary'].index
df_clean.drop(indexNames, inplace = True)

In [16]:
#Delete Retirement values with negatives
indexNames = df_clean.loc[df_clean['Retirement'] < 0, 'Retirement'].index
df_clean.drop(indexNames, inplace = True)

In [17]:
#Delete Health and Dental values with negatives
indexNames = df_clean.loc[df_clean['Health and Dental'] < 0, 'Health and Dental'].index
df_clean.drop(indexNames, inplace = True)

In [18]:
#Delete Other Benefits values with negatives
indexNames = df_clean.loc[df_clean['Other Benefits'] < 0, 'Other Benefits'].index
df_clean.drop(indexNames, inplace = True)

In [19]:
#Delete Total Benefits values with negatives
indexNames = df_clean.loc[df_clean['Total Benefits'] < 0, 'Total Benefits'].index
df_clean.drop(indexNames, inplace = True)

In [20]:
#Delete Total Compensation values with negatives
indexNames = df_clean.loc[df_clean['Total Compensation'] < 0, 'Total Compensation'].index
df_clean.drop(indexNames, inplace = True)

### Showing that there are no more negatives

In [21]:
df_clean.describe()

,Year,Organization Group Code,Union Code,Employee Identifier,Salaries,Overtime,Other Salaries,Total Salary,Retirement,Health and Dental,Other Benefits,Total Benefits,Total Compensation
count,333594.000000,333594.000000,333379.000000,3.335940e+05,333594.000000,333594.000000,333594.00000,333594.000000,333594.000000,333594.000000,333594.000000,333594.000000,333594.000000
mean,2016.278533,3.326253,489.103987,3.135046e+06,58551.935150,4465.978568,3184.66733,66202.581048,11465.227851,8199.214953,4364.253009,24028.695813,90231.276861
std,1.971718,1.890964,332.339140,4.112551e+06,46890.418363,11514.751974,6864.34306,54437.160996,9816.015221,5577.656978,4061.664248,17784.555424,71108.386635
min,2013.000000,1.000000,1.000000,1.000000e+00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2015.000000,2.000000,236.000000,2.232800e+04,12491.620000,0.000000,0.00000,13834.705000,347.620000,1663.670000,906.397500,4248.077500,18735.662500
50%,2017.000000,3.000000,535.000000,4.485350e+04,58749.380000,0.000000,455.70000,63487.825000,11776.875000,11258.665000,3812.365000,28709.615000,92064.485000
75%,2018.000000,5.000000,790.000000,8.526791e+06,89696.372500,2632.737500,3421.11250,100250.495000,18141.842500,12801.790000,6712.492500,37510.747500,137930.022500
max,2019.000000,7.000000,990.000000,1.071014e+07,631952.710000,309897.200000,336726.34000,637457.580000,118296.720000,22270.120000,37198.600000,141920.270000,779377.850000


### Fixes Union Code
In the 'Union Code' columns there were four categories of values that were null. By manually examining the data we found columns that already had the correct values so the code below inserts that correct information.

In [23]:
#=========Union Code is by Job=========
#Information Systems Trainee - 21, "Prof & Tech Eng, Local 21"
#Recreation Facility Assistant - 790, "SEIU, Local 1021, Misc"
#Technology Expert II - 351, "Municipal Exec Assoc, Misc"
#Transportation Controller Trainee - NONE, "NONE"
df_clean.loc[(df_clean['Job'] == "Information Systems Trainee") & (df_clean['Union Code'].isnull()), 'Union Code'] = 21
df_clean.loc[(df_clean['Job'] == "Information Systems Trainee") & (df_clean['Union'].isnull()), 'Union'] = "Prof & Tech Eng, Local 21"
#===================================================================================================================================================
df_clean.loc[(df_clean['Job'] == "Recreation Facility Assistant") & (df_clean['Union Code'].isnull()), 'Union Code'] = 790
df_clean.loc[(df_clean['Job'] == "Recreation Facility Assistant") & (df_clean['Union'].isnull()), 'Union'] = "SEIU, Local 1021, Misc"
#===================================================================================================================================================
df_clean.loc[(df_clean['Job'] == "Technology Expert II") & (df_clean['Union Code'].isnull()), 'Union Code'] = 351
df_clean.loc[(df_clean['Job'] == "Technology Expert II") & (df_clean['Union'].isnull()), 'Union'] = "Municipal Exec Assoc, Misc"
#===================================================================================================================================================
df_clean.loc[(df_clean['Job'] == "Transportation Controller Trainee") & (df_clean['Union Code'].isnull()), 'Union Code'] = 0
df_clean.loc[(df_clean['Job'] == "Transportation Controller Trainee") & (df_clean['Union'].isnull()), 'Union'] = "None"


### Missing Value Count
At this point most of the missing data has been dealt with. There is a little data that still needs to be cleaned.

In [24]:
# Check Missing Data
total = df_clean.isnull().sum().sort_values(ascending = False)
percent = (df_clean.isnull().sum() / df_clean.isnull().count() * 100).sort_values(ascending = False)
pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent']).transpose()

,Department,Department Code,Total Compensation,Total Benefits,Year,Organization Group Code,Organization Group,Union Code,Union,Job Family Code,...,Job,Employee Identifier,Salaries,Overtime,Other Salaries,Total Salary,Retirement,Health and Dental,Other Benefits,Year Type
Total,121254.000000,28940.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Percent,36.347776,8.675216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Fixing Job Family Code and Job Family using Job 
The two columns had null values "Unassigned" and "Untitled". Because the values of these columns is determined by the Job, the code below creates a dataframe containing every 'Job' category with its corresponding 'Job Family Code' and 'Job Family'. The block of code below this uses that dataframe to fix the 'Job Family Code' and 'Job Family' values that are null.

##### WARNING! RUNS FOR 7+ MINUTES

In [18]:
#Job Family Code = Job Family. They are determined by Job Code

#All the jobs
jobs = df_clean['Job Code'].unique()
dfj = pd.DataFrame(data=jobs)
dfj.columns = ['Job Code']

#Gets all unique groupings of Job Family/Code and Job Code
df_jobstuff = pd.DataFrame(columns=['Job Code','Job Family','Job Family Code'])
for index, row in dfj.iterrows():
    t = df_clean[ (df_clean['Job Code'] == row['Job Code']) & ((df_clean['Job Family'] != "Unassigned") & (df_clean['Job Family'] != "Untitled"))]
    arg = t.groupby(['Job Code', 'Job Family', 'Job Family Code']).size().reset_index(name = 'Freq')
    if arg['Job Code'].count() > 1:
        arg.head()
    arg = arg[['Job Code', 'Job Family', 'Job Family Code']]
    df_jobstuff = df_jobstuff.append(arg)
    
print("Done")

Done


##### Does the actual fixing of the unassigned and unknown values
##### WARNING! RUNS FOR 7+ MINUTES

In [19]:
df_clean = df_clean.reset_index(drop=True)

#Get a list of row ID's of ones missing stuff
errRowNums = df_clean.index[(df_clean['Job Family'] == "Unassigned") | (df_clean['Job Family'] == "Untitled")].tolist()
length = len(errRowNums)
for i in range(length): 
    jobname = df_clean.iloc[errRowNums[i]][10]
    r = df_jobstuff[df_jobstuff['Job Code'] == jobname]
    if r.empty:
        continue
    df_clean.loc[errRowNums[i], 'Job Family'] = r.iloc[0]['Job Family']
    df_clean.loc[errRowNums[i], 'Job Family Code'] = r.iloc[0]['Job Family Code']

print("Done!")


Done!


##### Saving currently done work 'df_clean' to a .csv file

In [24]:
df_clean.to_csv("EmpComp_Cleaned.csv", index = False, sep=',')
print("Done Saving File")

Done Saving File


##### Reading in the dataset that has been cleaned this far

In [2]:
#Read in the data from the file
df = pd.read_csv('C:/githubrepo/7331_Project/data/EmpComp_Cleaned.csv')
print("Finished Loading Data.")
df_clean = df

Finished Loading Data.


### Fixing the NULL 'Department' entries
The Department column is equivelent to the 'Department Code' column. The code below creates a dataframe containing every 'Department' category with its corresponding 'Department Code'. It then uses that dataframe to fix the majority of the null 'Department' values. 

### WARNING, TAKES ~15 MINUTES

In [10]:
dptcode = df_clean['Department Code'].unique()
df_dc = pd.DataFrame(data=dptcode)
df_dc.columns = ['Department Code']
df_dc = df_dc[df_dc['Department Code'].notnull()]

In [12]:
#Gets all unique groupings of Job Family/Code and Job Code
df_dptstuff = pd.DataFrame(columns=['Department Code','Department'])
for index, row in df_dc.iterrows():
    t = df_clean[ (df_clean['Department Code'] == row['Department Code']) & (df_clean['Department'].notnull())]
    arg = t.groupby(['Department Code', 'Department']).size().reset_index(name = 'Freq')
    if arg['Department Code'].count() > 1:
        break
    arg = arg[['Department Code', 'Department']]
    df_dptstuff = df_dptstuff.append(arg)
    
print("Done")

Done


In [16]:
#Making sure that it was saved
df_dptstuff.head()

,Department Code,Department
0,DPH,DPH Public Health
0,DPW,DPW GSA - Public Works
0,PUC,PUB Public Utilities Bureaus
0,ADM,ADM Gen Svcs Agency-City Admin
0,FIR,FIR Fire Department


In [21]:
df_clean = df_clean.reset_index(drop=True)

#Fixes the NULL Department values
errRowNums = df_clean.index[df_clean['Department'].isnull()].tolist()
length = len(errRowNums)
for i in range(length): 
    dptcode = df_clean.iloc[errRowNums[i]][4]
    r = df_dptstuff[df_dptstuff['Department Code'] == dptcode]
    if r.empty:
        continue
    df_clean.loc[errRowNums[i], 'Department'] = r.iloc[0]['Department']

print("Done!")

Done!


### Checking NULL's again
Most of the Department column is fixed, however 11% of them are still NULL.

In [22]:
# Check Missing Data
total = df_clean.isnull().sum().sort_values(ascending = False)
percent = (df_clean.isnull().sum() / df_clean.isnull().count() * 100).sort_values(ascending = False)
pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent']).transpose()

,Department,Department Code,Total Compensation,Total Benefits,Year,Organization Group Code,Organization Group,Union Code,Union,Job Family Code,...,Job,Employee Identifier,Salaries,Overtime,Other Salaries,Total Salary,Retirement,Health and Dental,Other Benefits,Year Type
Total,36936.000000,28940.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Percent,11.072142,8.675216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Saving currently done work 'df_clean' to a .csv file

In [25]:
df_clean.to_csv("EmpComp_Cleaned.csv", index = False, sep=',')
print("Done Saving File")

Done Saving File


##### Reading in the dataset that has been cleaned this far

In [9]:
df = pd.read_csv('C:/githubrepo/7331_Project/data/EmpComp_Cleaned.csv')
df_clean = df
print("Finished Loading Data.")

Finished Loading Data.


### Fixing the remaining 'Job Family' and 'Job Family Code' columns
After fixing these columns above there were still a four values that could not be automatically determined. By manually examinaning the data we figured out what these values should be. 

In [10]:
df_clean.loc[df_clean['Job Code'] == "0886", 'Job Family'] = "Management"
df_clean.loc[df_clean['Job Code'] == "0886", 'Job Family Code'] = "0900"
#======================================================================================================================================================
df_clean.loc[df_clean['Job Code'] == "381C", 'Job Family'] = "Legal and Count"
df_clean.loc[df_clean['Job Code'] == "381C", 'Job Family Code'] = "8100"
#======================================================================================================================================================
df_clean.loc[df_clean['Job Code'] == "274C", 'Job Family'] = "SF Superior Court"
df_clean.loc[df_clean['Job Code'] == "274C", 'Job Family Code'] = "SCRT"
#======================================================================================================================================================
df_clean.loc[df_clean['Job Code'] == "255C", 'Job Family'] = "SF Superior Court"
df_clean.loc[df_clean['Job Code'] == "255C", 'Job Family Code'] = "SCRT"

##### Saving currently done work 'df_clean' to a .csv file

In [11]:
df_clean.to_csv("EmpComp_Cleaned.csv", index = False, sep=',')
print("Done Saving File")

Done Saving File


##### Reading in the dataset that has been cleaned this far

In [13]:
df = pd.read_csv('C:/githubrepo/7331_Project/data/EmpComp_Cleaned.csv')
df_clean = df
print("Finished Loading Data.")

Finished Loading Data.


### Fixing Department Code blanks
In the 'Department Code' column there were some NULL values. These all had the same 'Organization Group' (which is a superset of the 'Department Code'). Because of this we created a new department code 'GCR'.

In [14]:
#Final remaining missing Department Codes, using abbrevation of Organization Group
df_clean.loc[df_clean['Department Code'].isnull(), 'Department Code'] = "GCR"

### Final Cleaned Dataset

In [17]:
df_clean.to_csv("EmpComp_Cleaned.csv", index = False, sep=',')
print("Done Saving File")

Done Saving File
